In [ ]:
from google.colab import drive
drive.mount( '/content/drive' )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error



In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.models import load_model

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))

In [ ]:

def split_x(size, obs):
    x = []

    for i in range(len(obs)-SEQUENCE_SIZE):
        window = obs[i:(i+SEQUENCE_SIZE)]
        window = [[x] for x in window]
        x.append(window)

    return np.array(x)


In [ ]:
def split_y(size, obs):
    y = []

    for i in range(len(obs)-SEQUENCE_SIZE):
        after_window = obs[i+SEQUENCE_SIZE]
        y.append(after_window)

    return np.array(y)

# **...... BITCOIN......**

In [ ]:
btcpath='/content/BTC-USD (6).csv'


In [ ]:
btc=pd.read_csv(btcpath)

In [ ]:
btc

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-01,963.658020,1003.080017,958.698975,998.325012,998.325012,147775008
1,2017-01-02,998.617004,1031.390015,996.702026,1021.750000,1021.750000,222184992
2,2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,1043.839966,185168000
3,2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,1154.729980,344945984
4,2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,1013.380005,510199008
...,...,...,...,...,...,...,...
2474,2023-10-11,27392.076172,27474.115234,26561.099609,26873.320313,26873.320313,13648094333
2475,2023-10-12,26873.292969,26921.439453,26558.320313,26756.798828,26756.798828,9392909295
2476,2023-10-13,26752.878906,27092.697266,26686.322266,26862.375000,26862.375000,15165312851
2477,2023-10-14,26866.203125,26969.000000,26814.585938,26861.707031,26861.707031,5388116782


In [ ]:
btc_features =btc.iloc[:,1:5]

In [ ]:
btc_features

,Open,High,Low,Close
0,963.658020,1003.080017,958.698975,998.325012
1,998.617004,1031.390015,996.702026,1021.750000
2,1021.599976,1044.079956,1021.599976,1043.839966
3,1044.400024,1159.420044,1044.400024,1154.729980
4,1156.729980,1191.099976,910.416992,1013.380005
...,...,...,...,...
2474,27392.076172,27474.115234,26561.099609,26873.320313
2475,26873.292969,26921.439453,26558.320313,26756.798828
2476,26752.878906,27092.697266,26686.322266,26862.375000
2477,26866.203125,26969.000000,26814.585938,26861.707031


In [ ]:
btc_features=scaler.fit_transform(btc_features)

In [ ]:
btc_features

array([[0.00282263, 0.00264503, 0.0030924 , 0.00330246],
       [0.00334617, 0.00306156, 0.00367149, 0.00365319],
       [0.00369036, 0.00324827, 0.00405088, 0.00398393],
       ...,
       [0.38903592, 0.38650601, 0.39512457, 0.39055219],
       [0.39073304, 0.38468603, 0.39707903, 0.39054219],
       [0.39052838, 0.38623614, 0.39720766, 0.39298224]])

In [ ]:
btc_close=btc.iloc[:,4:5]
btc_close=scaler.fit_transform(btc_close)


In [ ]:
#splite X_train and x_test using all features.........

btc_train_size = int(len(btc_features)*.8 )
btc_train=btc_features[:btc_train_size]
btc_test=btc_features[btc_train_size:]


In [ ]:
SEQUENCE_SIZE=30
btcx_train= split_x(SEQUENCE_SIZE,btc_train)
btcx_test = split_x(SEQUENCE_SIZE,btc_test)

print("Shape of training set: {}".format(btcx_train.shape))


Shape of training set: (1953, 30, 1, 4)


In [ ]:
# splite y_train and y_test using Close only.......

btc_train_size_close = int(len(btc_close)*.8 )
btc_train_close=btc_close[:btc_train_size_close]
btc_test_close=btc_close[btc_train_size_close:]

In [ ]:
btcy_train= split_y(SEQUENCE_SIZE,btc_train_close)
btcy_test = split_y(SEQUENCE_SIZE,btc_test_close)

In [ ]:
btcx_train.shape


(1953, 30, 1, 4)

In [ ]:
btcx_test.shape

(466, 30, 1, 4)

In [ ]:
btcy_train.shape

(1953, 1)

In [ ]:
btcy_test.shape

(466, 1)

In [ ]:
btcx_train = btcx_train.reshape( btcx_train.shape[0], btcx_train.shape[1],4)
btcx_test = btcx_test.reshape( btcx_test.shape[0], btcx_test.shape[1],4)

In [ ]:
btc_train

array([[0.00282263, 0.00264503, 0.0030924 , 0.00330246],
       [0.00334617, 0.00306156, 0.00367149, 0.00365319],
       [0.00369036, 0.00324827, 0.00405088, 0.00398393],
       ...,
       [0.43326324, 0.42826003, 0.4379989 , 0.43502861],
       [0.43519479, 0.43101404, 0.43913325, 0.43613281],
       [0.43632046, 0.45419533, 0.44400535, 0.45805271]])

In [ ]:
btcx_train

array([[[0.00282263, 0.00264503, 0.0030924 , 0.00330246],
        [0.00334617, 0.00306156, 0.00367149, 0.00365319],
        [0.00369036, 0.00324827, 0.00405088, 0.00398393],
        ...,
        [0.00216599, 0.0014802 , 0.00249984, 0.00215354],
        [0.00219978, 0.00147295, 0.00248973, 0.00212219],
        [0.00217108, 0.00146751, 0.0024947 , 0.00213545]],

       [[0.00334617, 0.00306156, 0.00367149, 0.00365319],
        [0.00369036, 0.00324827, 0.00405088, 0.00398393],
        [0.00403181, 0.00494529, 0.0043983 , 0.00564423],
        ...,
        [0.00219978, 0.00147295, 0.00248973, 0.00212219],
        [0.00217108, 0.00146751, 0.0024947 , 0.00213545],
        [0.00218318, 0.00218802, 0.00251733, 0.00288439]],

       [[0.00369036, 0.00324827, 0.00405088, 0.00398393],
        [0.00403181, 0.00494529, 0.0043983 , 0.00564423],
        [0.00571403, 0.0054114 , 0.00235669, 0.00352787],
        ...,
        [0.00217108, 0.00146751, 0.0024947 , 0.00213545],
        [0.00218318, 0.002188

In [ ]:
btcy_train

array([[0.00288439],
       [0.00316318],
       [0.00350421],
       ...,
       [0.43502861],
       [0.43613281],
       [0.45805271]])

In [ ]:
btcx_train.shape

(1953, 30, 4)

In [ ]:
btcx_test.shape

(466, 30, 4)

In [ ]:
model=Sequential()
model.add(LSTM(units=30,return_sequences=True,input_shape=(btcx_train.shape[1],4)))
model.add(Dropout(0.2))
model.add(LSTM(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=90,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=120,return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 30, 30)              │           4,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 30, 30)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 30, 60)              │          21,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 30, 60)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 30, 90)              │          54,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 30, 90)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 120)                 │         101,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             121 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 181,801 (710.16 KB)

 Trainable params: 181,801 (710.16 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(btcx_train,btcy_train,epochs=100,batch_size=32)

Epoch 1/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 11s 77ms/step - loss: 0.0195
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 7s 106ms/step - loss: 0.0038
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - loss: 0.0036
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 7s 100ms/step - loss: 0.0025
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.0021
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - loss: 0.0021
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 8s 78ms/step - loss: 0.0077
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - loss: 0.0018
Epoch 9/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - loss: 0.0022
Epoch 10/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 7s 116ms/step - loss: 0.0047
Epoch 11/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 8s 80ms/step - loss: 0.0019
Epoch 12/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 8s 121ms/step - loss: 0.0015
Epoch 13/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 8s 78ms/step - loss: 0.0021
Epoch 14/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 7s 115ms/step - loss: 0.0017
Epoch 15/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 8s 78ms/s

In [ ]:
btcy_test.shape

(466, 1)

In [ ]:
btcy_pred=model.predict(btcx_test)

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step


In [ ]:
btcy_pred.shape

(466, 1)

In [ ]:
model.save('lstm_btc.h5')

In [ ]:
model = load_model('lstm_btc.h5')

In [ ]:
btcy_test.shape

(466, 1)

In [ ]:

print('MAE =',mean_absolute_error(btcy_test ,btcy_pred))


MAE = 0.008020557987893967


In [ ]:
print('MAPE =',mean_absolute_percentage_error(btcy_test,btcy_pred))

MAPE = 0.023623512523775696


In [ ]:
print('MSE=',mean_squared_error(btcy_test,btcy_pred))

MSE= 0.0001258097288616405


In [ ]:
#Root Mean Sqquare Erorr
print('RMSE=',np.sqrt(mean_squared_error(btcy_test,btcy_pred)))

RMSE= 0.011216493608148694


# **......LETCOIN......**

In [ ]:
ltcpath='/content/LTC-USD (1).csv'


In [ ]:
ltc=pd.read_csv(ltcpath)

In [ ]:
ltc

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-18,5.065430,5.065430,4.579960,4.685230,4.685230,4569260
1,2014-09-19,4.687290,4.755820,4.254350,4.327770,4.327770,3917450
2,2014-09-20,4.329200,4.616080,4.202190,4.286440,4.286440,5490660
3,2014-09-21,4.263070,4.300130,4.154990,4.245920,4.245920,2931220
4,2014-09-22,4.245930,4.416880,4.210130,4.242350,4.242350,1855960
...,...,...,...,...,...,...,...
3310,2023-10-11,63.619778,63.685921,61.262772,61.499821,61.499821,186389594
3311,2023-10-12,61.501141,61.521873,60.754681,61.127815,61.127815,171888213
3312,2023-10-13,61.127197,61.995136,61.112534,61.574059,61.574059,203849754
3313,2023-10-14,61.574585,61.949982,61.470036,61.665157,61.665157,120785909


In [ ]:
ltc_features =ltc.iloc[:,1:5]

In [ ]:
ltc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3315 entries, 0 to 3314
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3315 non-null   object 
 1   Open       3315 non-null   float64
 2   High       3315 non-null   float64
 3   Low        3315 non-null   float64
 4   Close      3315 non-null   float64
 5   Adj Close  3315 non-null   float64
 6   Volume     3315 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 181.4+ KB


In [ ]:
ltc_features=scaler.fit_transform(ltc_features)

In [ ]:
ltc_features

array([[0.01011644, 0.00903907, 0.0100708 , 0.00915722],
       [0.00913862, 0.00828689, 0.00912477, 0.00822946],
       [0.00821264, 0.0079474 , 0.00897322, 0.00812219],
       ...,
       [0.1550853 , 0.1473471 , 0.17432131, 0.15680775],
       [0.15624219, 0.1472374 , 0.17536   , 0.15704419],
       [0.15652653, 0.14716836, 0.17531297, 0.15729666]])

In [ ]:
ltc_close=ltc.iloc[:,4:5]
ltc_close=scaler.fit_transform(ltc_close)


In [ ]:
#splite X_train and x_test using all features.........

ltc_train_size = int(len(ltc_features)*.8 )
ltc_train=ltc_features[:ltc_train_size]
ltc_test=ltc_features[ltc_train_size:]


In [ ]:
SEQUENCE_SIZE=30
ltcx_train= split_x(SEQUENCE_SIZE,ltc_train)
ltcx_test = split_x(SEQUENCE_SIZE,ltc_test)


print("Shape of training set: {}".format(ltcx_train.shape))

Shape of training set: (2622, 30, 1, 4)


In [ ]:
# splite y_train and y_test using Close only.......

ltc_train_size_close = int(len(ltc_close)*.8 )
ltc_train_close=ltc_close[:ltc_train_size_close]
ltc_test_close=ltc_close[ltc_train_size_close:]

In [ ]:
ltcy_train= split_y(SEQUENCE_SIZE,ltc_train_close)
ltcy_test = split_y(SEQUENCE_SIZE,ltc_test_close)

In [ ]:
ltcx_train.shape


(2622, 30, 1, 4)

In [ ]:
ltcx_test.shape

(633, 30, 1, 4)

In [ ]:
ltcy_train.shape

(2622, 1)

In [ ]:
ltcy_test.shape

(633, 1)

In [ ]:
ltcx_train = ltcx_train.reshape( ltcx_train.shape[0], ltcx_train.shape[1], 4)
ltcx_test = ltcx_test.reshape( ltcx_test.shape[0], ltcx_test.shape[1], 4)

In [ ]:
ltcx_test.shape

(633, 30, 4)

In [ ]:
ltcx_train.shape

(2622, 30, 4)

In [ ]:
model=Sequential()
model.add(LSTM(units=30,return_sequences=True,input_shape=(ltcx_train.shape[1],4)))
model.add(Dropout(0.2))
model.add(LSTM(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 30, 30)              │           4,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 30, 30)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 30, 60)              │          21,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 30, 60)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 90)                  │          54,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 90)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              91 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 80,491 (314.42 KB)

 Trainable params: 80,491 (314.42 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(ltcx_train,ltcy_train,epochs=100,batch_size=30)

Epoch 1/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - loss: 0.0072
Epoch 2/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 0.0029
Epoch 3/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0019
Epoch 4/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - loss: 0.0019
Epoch 5/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - loss: 0.0019
Epoch 6/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.0013
Epoch 7/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 0.0015
Epoch 8/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - loss: 0.0014
Epoch 9/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.0015
Epoch 10/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 9.8118e-04
Epoch 11/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - loss: 0.0017
Epoch 12/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0011
Epoch 13/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - loss: 0.0012
Epoch 14/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - loss: 0.0011
Epoch 15/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step 

In [ ]:
ltcy_pred=model.predict(ltcx_test)

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step


In [ ]:
model.save('lstm_ltc.h5')

In [ ]:
model = load_model('lstm_ltc.h5')

In [ ]:

print('MAE =',mean_absolute_error(ltcy_test,ltcy_pred))


MAE = 0.00851176329798628


In [ ]:
print('MAPE =',mean_absolute_percentage_error(ltcy_test,ltcy_pred))

MAPE = 0.04396314054907952


In [ ]:
print('MSE=',mean_squared_error(ltcy_test,ltcy_pred))

MSE= 0.00012291478623196341


In [ ]:
#Root Mean Squre Erorr
print('RMSE=',np.sqrt(mean_squared_error(ltcy_test,ltcy_pred)))

RMSE= 0.011086694107440839


# **......ETHEREUM......**

In [ ]:
ethpath='/content/ETH-USD.csv'


In [ ]:
eth=pd.read_csv(ethpath)

In [ ]:
eth

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-12-01,445.209015,472.609009,428.312012,466.540009,466.540009,1247879936
1,2017-12-02,466.851013,476.239014,456.653015,463.449005,463.449005,943649984
2,2017-12-03,463.704987,482.813995,451.851990,465.852997,465.852997,990556992
3,2017-12-04,466.053986,474.777008,453.312012,470.204010,470.204010,1005550016
4,2017-12-05,470.294006,473.558014,457.660004,463.281006,463.281006,1216720000
...,...,...,...,...,...,...,...
2140,2023-10-11,1567.680786,1578.223511,1548.980469,1566.254761,1566.254761,5416504273
2141,2023-10-12,1566.355957,1566.878174,1523.237549,1539.612427,1539.612427,5003930677
2142,2023-10-13,1539.432861,1571.750610,1537.921143,1552.089478,1552.089478,4575141511
2143,2023-10-14,1552.263794,1560.325073,1545.738770,1555.256836,1555.256836,2429214718


In [ ]:
eth_features=eth.iloc[: , 1:5]

In [ ]:
eth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145 entries, 0 to 2144
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2145 non-null   object 
 1   Open       2145 non-null   float64
 2   High       2145 non-null   float64
 3   Low        2145 non-null   float64
 4   Close      2145 non-null   float64
 5   Adj Close  2145 non-null   float64
 6   Volume     2145 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 117.4+ KB


In [ ]:

eth_features=scaler.fit_transform(eth_features)

In [ ]:
eth_close=eth.iloc[:,4:5]
eth_close=scaler.fit_transform(eth_close)


In [ ]:
#splite X_train and x_test using all features.........

eth_train_size = int(len(eth_features)*.8 )

eth_train=eth_features[:eth_train_size]
eth_test=eth_features[eth_train_size:]

In [ ]:
SEQUENCE_SIZE=30
ethx_train= split_x(SEQUENCE_SIZE,eth_train)
ethx_test = split_x(SEQUENCE_SIZE,eth_test)

print("Shape of training set: {}".format(ethx_train.shape))


Shape of training set: (1686, 30, 1, 4)


In [ ]:
# splite y_train and y_test using Close only.......

eth_train_size_close = int(len(eth_close)*.8 )

eth_train_close=eth_close[:eth_train_size_close]
eth_test_close=eth_close[eth_train_size_close:]

In [ ]:
ethy_train= split_y(SEQUENCE_SIZE,eth_train_close)
ethy_test = split_y(SEQUENCE_SIZE,eth_test_close)

In [ ]:
ethx_train.shape


(1686, 30, 1, 4)

In [ ]:
ethx_test.shape

(399, 30, 1, 4)

In [ ]:
ethy_train.shape

(1686, 1)

In [ ]:
ethy_test.shape

(399, 1)

In [ ]:
ethx_train = ethx_train.reshape( ethx_train.shape[0], ethx_train.shape[1], 4)
ethx_test = ethx_test.reshape( ethx_test.shape[0], ethx_test.shape[1], 4)

In [ ]:
ethx_test.shape

(399, 30, 4)

In [ ]:
ethx_train.shape

(1686, 30, 4)

In [ ]:
model=Sequential()
model.add(LSTM(units=30,return_sequences=True,input_shape=(ethx_train.shape[1],4)))
model.add(Dropout(0.2))
model.add(LSTM(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                        │ (None, 30, 30)              │           4,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 30, 30)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 30, 60)              │          21,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 30, 60)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 90)                  │          54,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 90)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              91 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 80,491 (314.42 KB)

 Trainable params: 80,491 (314.42 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(ethx_train,ethy_train,epochs=100,batch_size=30)

Epoch 1/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - loss: 0.0281
Epoch 2/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.0034
Epoch 3/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0039
Epoch 4/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0029
Epoch 5/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 0.0031
Epoch 6/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0027
Epoch 7/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0025
Epoch 8/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 0.0031
Epoch 9/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0025
Epoch 10/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - loss: 0.0024
Epoch 11/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.0020
Epoch 12/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.0020
Epoch 13/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.0017
Epoch 14/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - loss: 0.0021
Epoch 15/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 77ms/step - lo

In [ ]:
ethy_pred=model.predict(ethx_test)

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step


In [ ]:
model.save('lstm_eth.h5')

In [ ]:
model = load_model('lstm_eth.h5')

In [ ]:
print('MAE =',mean_absolute_error(ethy_test,ethy_pred))


MAE = 0.007795488346925301


In [ ]:
print('MAPE =',mean_absolute_percentage_error(ethy_test,ethy_pred))

MAPE = 0.02463157677583696


In [ ]:
print('MSE=',mean_squared_error(ethy_test,ethy_pred))

MSE= 0.00012217727932798477


In [ ]:
#Root Mean Square Erorre
print('RMSE=',np.sqrt(mean_squared_error(ethy_test,ethy_pred)))

RMSE= 0.011053383162090454


# **......XMR.....**

In [ ]:
xmr_path='/content/XMR-USD.csv'

In [ ]:
xmr=pd.read_csv(xmr_path)

In [ ]:
xmr

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-11-10,121.344002,121.665001,101.757004,105.585999,105.585999,84614000
1,2017-11-11,105.750000,127.106003,103.877998,119.615997,119.615997,107708000
2,2017-11-12,119.597000,133.675003,110.617996,123.856003,123.856003,144948000
3,2017-11-13,128.960007,136.528000,120.921997,123.402000,123.402000,116200000
4,2017-11-14,123.615997,124.200996,118.625999,122.352997,122.352997,53544800
...,...,...,...,...,...,...,...
2161,2023-10-11,152.220093,153.929443,150.970428,153.842102,153.842102,61162383
2162,2023-10-12,153.852783,154.344238,151.021347,152.341812,152.341812,65445853
2163,2023-10-13,152.339523,154.368774,151.051971,153.590515,153.590515,51282283
2164,2023-10-14,153.583801,154.282990,151.850967,152.833160,152.833160,33344438


In [ ]:
xmr_features=xmr.iloc[: , 1:5]

In [ ]:
xmr_features

,Open,High,Low,Close
0,121.344002,121.665001,101.757004,105.585999
1,105.750000,127.106003,103.877998,119.615997
2,119.597000,133.675003,110.617996,123.856003
3,128.960007,136.528000,120.921997,123.402000
4,123.615997,124.200996,118.625999,122.352997
...,...,...,...,...
2161,152.220093,153.929443,150.970428,153.842102
2162,153.852783,154.344238,151.021347,152.341812
2163,152.339523,154.368774,151.051971,153.590515
2164,153.583801,154.282990,151.850967,152.833160


In [ ]:
xmr_features=scaler.fit_transform(xmr_features)

In [ ]:
xmr_features

array([[0.19585978, 0.17676522, 0.17589732, 0.16107407],
       [0.16125454, 0.18807766, 0.18086816, 0.19221218],
       [0.19198294, 0.20173534, 0.19666427, 0.20162242],
       ...,
       [0.26464313, 0.24475998, 0.29142686, 0.26761504],
       [0.26740435, 0.24458163, 0.29329942, 0.26593417],
       [0.26562223, 0.24288277, 0.29405913, 0.26592798]])

In [ ]:
xmr_close=xmr.iloc[:,4:5]
xmr_close=scaler.fit_transform(xmr_close)


In [ ]:
#splite X_train and x_test using all features.........

xmr_train_size = int(len(xmr_features)*.8 )

xmr_train=xmr_features[:xmr_train_size]
xmr_test=xmr_features[xmr_train_size:]

In [ ]:
SEQUENCE_SIZE=30
xmrx_train= split_x(SEQUENCE_SIZE,xmr_train)
xmrx_test = split_x(SEQUENCE_SIZE,xmr_test)

print("Shape of training set: {}".format(xmrx_train.shape))


Shape of training set: (1702, 30, 1, 4)


In [ ]:
# splite y_train and y_test using Close only.......

xmr_train_size_close = int(len(xmr_close)*.8 )

xmr_train_close=xmr_close[:xmr_train_size_close]
xmr_test_close=xmr_close[xmr_train_size_close:]

In [ ]:
xmry_train= split_y(SEQUENCE_SIZE,xmr_train_close)
xmry_test = split_y(SEQUENCE_SIZE,xmr_test_close)

In [ ]:
xmrx_train.shape


(1702, 30, 1, 4)

In [ ]:
xmrx_test.shape

(404, 30, 1, 4)

In [ ]:
xmry_train.shape

(1702, 1)

In [ ]:
xmry_test.shape

(404, 1)

In [ ]:
xmrx_train = xmrx_train.reshape( xmrx_train.shape[0], xmrx_train.shape[1], 4)
xmrx_test = xmrx_test.reshape( xmrx_test.shape[0], xmrx_test.shape[1], 4)

In [ ]:
xmrx_test.shape

(404, 30, 4)

In [ ]:
xmrx_train.shape

(1702, 30, 4)

In [ ]:
model=Sequential()
model.add(LSTM(units=30,return_sequences=True,input_shape=(xmrx_train.shape[1],4)))
model.add(Dropout(0.2))
model.add(LSTM(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                       │ (None, 30, 30)              │           4,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 30, 30)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_11 (LSTM)                       │ (None, 30, 60)              │          21,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 30, 60)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_12 (LSTM)                       │ (None, 90)                  │          54,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 90)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              91 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 80,491 (314.42 KB)

 Trainable params: 80,491 (314.42 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(xmrx_train,xmry_train,epochs=100,batch_size=30)

In [ ]:
xmry_pred=model.predict(xmrx_test)

In [ ]:
model.save('lstm_xmr.h5')

In [ ]:
model = load_model('lstm_xmr.h5')

In [ ]:
from sklearn.metrics import mean_absolute_error
print('MAE =',mean_absolute_error(xmry_test,xmry_pred))


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
print('MAPE =',mean_absolute_percentage_error(xmry_test,xmry_pred))

In [ ]:
from sklearn.metrics import mean_squared_error
print('MSE=',mean_squared_error(xmry_test,xmry_pred))

In [ ]:
#Root Mean Square Erorre
print('RMSE=',np.sqrt(mean_squared_error(xmry_test,xmry_pred)))